In [ ]:
%load_ext autoreload
%autoreload 2

import json
import os
import pickle
from multiprocessing import Pool
from pathlib import Path
from time import gmtime, strftime, time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from utils import TPScoringFunction, calc_auc, ecfp, score

def timestamp():
    return strftime("%Y-%m-%d_%H:%M:%S", gmtime())

def fit_clfs(chid, n_estimators, n_jobs):
    """
    Args:
        chid: which assay to use:
        external_file:
    """
    # read data and calculate ecfp fingerprints
    assay_file = f'./assays/processed/{chid}.csv'
    print(f'Reading data from: {assay_file}')
    df = pd.read_csv(assay_file)
    X = np.array(ecfp(df.smiles))
    y = np.array(df.label)

    # split in equally sized sets. Stratify to get same label distributions
    X1, X2, y1, y2 = train_test_split(X, y, test_size=0.5, stratify=y)

    balance = (np.mean(y1), np.mean(y2))

    # train classifiers and store them in dictionary
    clfs = {}
    clfs['Split1'] = RandomForestClassifier(
        n_estimators=n_estimators, n_jobs=n_jobs)
    clfs['Split1'].fit(X1, y1)

    clfs['Split1_alt'] = RandomForestClassifier(
        n_estimators=n_estimators, n_jobs=n_jobs)
    clfs['Split1_alt'].fit(X1, y1)

    clfs['Split2'] = RandomForestClassifier(
        n_estimators=n_estimators, n_jobs=n_jobs)
    clfs['Split2'].fit(X2, y2)

    # calculate AUCs for the clfs
    aucs = {}
    aucs['Split1'] = calc_auc(clfs['Split1'], X2, y2)
    aucs['Split1_alt'] = calc_auc(clfs['Split1_alt'], X2, y2)
    aucs['Split2'] = calc_auc(clfs['Split2'], X1, y1)
    print("AUCs:")
    for k, v in aucs.items():
        print(f'{k}: {v}')

    return clfs, aucs, balance

In [ ]:
# def optimize(chid,
#              n_estimators,
#              n_jobs,
#              external_file,
#              n_external,
#              seed,
#              optimizer_args):


chid='CHEMBL3888429'
n_estimators=100
n_jobs=8
external_file='./data/guacamol_v1_test.smiles.can'
n_external=3000
seed=101

    
np.random.seed(seed)
# config = locals()
# print(locals())
#set up logging
results_dir = os.path.join('./test', 'graph_ga', chid, timestamp())
os.makedirs(results_dir)

# config_file = os.path.join(results_dir, 'config.json')
# with open(config_file, 'w') as f:
#     json.dump(config, f)



clfs, aucs, balance = fit_clfs(chid, n_estimators, n_jobs)
results = {}
results['AUC'] = aucs
results['balance'] = balance

clf_file = os.path.join(results_dir, 'classifiers.p')
with open(clf_file, 'wb') as f:
    pickle.dump(clfs, f)

Reading data from: ./assays/processed/CHEMBL3888429.csv
AUCs:
Split1: 0.8218614718614718
Split1_alt: 0.8124458874458874
Split2: 0.8033948940793046


In [ ]:
for k, v in optimizer_args.items():
    print(f'{k} = {repr(v)},')

pretrained_model_path = 'hi',
n_epochs = 4,
mols_to_sample = 1028,
keep_top = 512,
optimize_n_epochs = 2,
max_len = 100,
optimize_batch_size = 64,
number_final_samples = 1028,
sample_final_model_only = False,
random_start = False,
smi_file = None,
n_jobs = -1,


In [ ]:
from generators import GB_GA_Generator, SmilesRnnDirectedGenerator

# Create guacamol scoring function with clf trained on split 1
scoring_function = TPScoringFunction(clfs['Split1'])

optimizer_args = dict(pretrained_model_path = './guacamol_baselines/smiles_lstm_hc/pretrained_model/model_final_0.473.pt',
                        n_epochs = 4,
                        mols_to_sample = 1028,
                        keep_top = 512,
                        optimize_n_epochs = 2,
                        max_len = 100,
                        optimize_batch_size = 64,
                        number_final_samples = 1028,
                        sample_final_model_only = False,
                        random_start = True,
                        smi_file = './data/guacamol_v1_train.smiles.can',
                        n_jobs = -1,
                        canonicalize=False)

# run optimization
t0 = time()
optimizer = SmilesRnnDirectedGenerator(**optimizer_args)
smiles_history = optimizer.generate_optimized_molecules(scoring_function, 100, get_history=True)

selecting initial population...


In [ ]:
len(smiles_history)

'abcccc' < 'bcc'

True

In [ ]:
# make a list of dictionaries for every time step
statistics = []
for optimized_smiles in smiles_history:
    row = {}
    row['smiles'] = optimized_smiles
    row['preds'] = {}
    row['ratio_active'] = {}
    row['mean_pred'] = {}
    for k, clf in clfs.items():
        preds = score(optimized_smiles, clf)
        row['preds'][k] = preds
    statistics.append(row)

results['statistics'] = statistics

stat_time = time() - t1
# add predictions on external set
# load external smiles for evaluation
with open(external_file) as f:
    external_smiles = f.read().split()
external_smiles = np.random.choice(external_smiles, n_external)
results['predictions_external'] = {k: score(external_smiles, clf) for k, clf in clfs.items()}

results_file = os.path.join(results_dir, 'results.json')
with open(results_file, 'w') as f:
    json.dump(results, f)

print(f'Storing results in {results_dir}')
print(f'Optimization time {opt_time:.2f}')
print(f'Statistics time {stat_time:.2f}')

In [ ]:
import argparse
parser = argparse.ArgumentParser(description='Goal-directed generation benchmark for SMILES RNN',
                                     formatter_class=argparse.ArgumentDefaultsHelpFormatter)
parser.add_argument('--model_path', default=None, help='Full path to the pre-trained SMILES RNN model')
parser.add_argument('--max_len', default=100, type=int, help='Max length of a SMILES string')
parser.add_argument('--seed', default=42, type=int, help='Random seed')
parser.add_argument('--output_dir', default=None, help='Output directory for results')
parser.add_argument('--number_repetitions', default=1, type=int, help='Number of re-training runs to average')
parser.add_argument('--keep_top', default=512, type=int, help='Molecules kept each step')
parser.add_argument('--n_epochs', default=20, type=int, help='Epochs to sample')
parser.add_argument('--mols_to_sample', default=1024, type=int, help='Molecules sampled at each step')
parser.add_argument('--optimize_batch_size', default=256, type=int, help='Batch size for the optimization')
parser.add_argument('--optimize_n_epochs', default=2, type=int, help='Number of epochs for the optimization')
parser.add_argument('--benchmark_num_samples', default=4096, type=int,
                    help='Number of molecules to generate from final model for the benchmark')
parser.add_argument('--benchmark_trajectory', action='store_true',
                    help='Take molecules generated during re-training into account for the benchmark')
parser.add_argument('--smiles_file', default='data/guacamol_v1_all.smiles')
parser.add_argument('--random_start', action='store_true')
parser.add_argument('--n_jobs', type=int, default=-1)
parser.add_argument('--suite', default='v2')

args = parser.parse_args('')
for k,v in args.__dict__.items():
    print(f'{k} = {v},')
    
